In [1]:
import tensorflow_datasets as tfds

dataset, info = tfds.load(
    'cats_vs_dogs',
    split='train',
    as_supervised=True,
    with_info=True
)

print(info)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.Q5XXB3_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/root/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings 

In [2]:
import numpy as np
import cv2

X = []
y = []

IMG_SIZE = 64

for image, label in dataset.take(2000):   # 2000 samples enough for SVM
    img = cv2.resize(image.numpy(), (IMG_SIZE, IMG_SIZE))
    img = img.flatten()
    X.append(img)
    y.append(label.numpy())

X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)


(2000, 12288) (2000,)


In [5]:
import numpy as np
import cv2

IMG_SIZE = 64

X = []
y = []

count = 0

for image, label in dataset:
    img = image.numpy()
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    X.append(img.flatten())
    y.append(label.numpy())

    count += 1
    if count == 2000:
        break

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (2000, 12288)
y shape: (2000,)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)


Train shape: (1600, 12288) (1600,)
Test shape: (400, 12288) (400,)


In [7]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

print("SVM training completed")


SVM training completed


In [8]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Accuracy: 0.5575

Classification Report:

              precision    recall  f1-score   support

           0       0.55      0.61      0.58       198
           1       0.57      0.50      0.54       202

    accuracy                           0.56       400
   macro avg       0.56      0.56      0.56       400
weighted avg       0.56      0.56      0.56       400

